## Run on Mock Data Challenge

This notebook looks at how to run state-space algorithms on data from the [IPTA Mock Data Challenge](https://web.archive.org/web/20130108011819/http://www.ipta4gw.org/?page_id=126). See also [https://github.com/nanograv/mdc1](https://github.com/nanograv/mdc1) and https://github.com/ipta/mdc2


It uses some of the methods in `explore_how_minnow_works.ipynb`, but uses exclusively single band data (i.e. TOAs recorded at a single observing radio frequency), in keeping with the single band data provided by the MDC.

---

For all that follows we will assume that there is just a single pulsar, $N_{\rm psr} =1$. The extension to general $N_{\rm psr}$ should be straightforward. 

For the purposes of running state-space algorithms, we need to define the following:

* $\boldsymbol{X}$ : the state vector, dimension $n_X$
* $\boldsymbol{Y}$ : the observation vector, dimension $n_Y$
* $\boldsymbol{F}$ : the state transition matrix, dimension $n_X$ $\times$ $n_X$
* $\boldsymbol{H}$ : the measurement matrix, dimension $n_Y$ $\times$ $n_X$
* $\boldsymbol{Q}$ : the process noise covariance matrix, dimension $n_X$ $\times$ $n_X$
* $\boldsymbol{R}$ : the measurement noise covariance matrix, dimension $n_Y$ $\times$ $n_Y$

### 1. Observations

The 'raw' pulsar data comes in the form of a `.toa` file and a `.par` file.

The `.toa` gives you the pulse times of arrival, the `.par` file gives the best guess of some of the pulsar parameters such as its spin frequency, position on the sky, etc. Note that these parameters are very well known a-priori. 

The `.toa` and the `.par` get passed through a timing software like `TEMPO` or `PINT` to produce **timing residuals**

The class `SingleBandPulsar` below is copied verbatim from `minnow`. It provides an interface to loading a `par` and `tim` file and producing timing residuals. In the example below we use `PINT`. 

**Question 1:** are the values of `ephem`,`bipm_version`,and `clk` correct?


In [1]:
import numpy as np 
from enterprise.pulsar import Pulsar as enterprise_Pulsar


class SingleBandPulsar(): #note: I have removed the ds.Pulsar argument which depends on the Discovery package. not needed here
    """Single -- A class for handling pulsar data with a single frequency channels
    at given TOA.
    """
    def __init__(self, toas, residuals, radio_frequencies,
                 toaerrs, backend_flags, Mmat, fitpars,
                 noisedict=None, name='psr'):

        self.toas = toas
        self.toaerrs = toaerrs
        self.residuals = residuals
        self.radio_freqs = radio_frequencies
        self.backend_flags = backend_flags
        self.fitpars = fitpars

        self.toa_diffs = np.diff(toas)
        self.toa_diff_errors = np.sqrt(toaerrs[1:]**2 + toaerrs[:-1]**2)

        # cut up Mmat
        # Mmat = cutfunc(Mmat, fitpars)
        self.Mmat = Mmat


        if noisedict:
            self.noisedict = noisedict # we can probably cut this. State-space algos should handle the noise. 
        self.name = name


    @classmethod
    def read_par_tim(cls, p, t, **kwargs):
        return cls.from_enterprise(enterprise_Pulsar(str(p), str(t), **kwargs))

    @classmethod
    def from_enterprise(cls, ds_psr):

        if hasattr(ds_psr, 'noisedict'):
            noisedict = ds_psr.noisedict
        else:
            noisedict = None
        return cls(ds_psr.toas, ds_psr.residuals,
                   ds_psr.freqs, ds_psr.toaerrs,
                   ds_psr.backend_flags, ds_psr.Mmat, ds_psr.fitpars,
                   noisedict=noisedict, name=ds_psr.name)

In [2]:

psr_name = 'J0030+0451'

par = f'../data/IPTA_MockDataChallenge/IPTA_Challenge1_open/Challenge_Data/Dataset1/{psr_name}.par'
tim = f'../data/IPTA_MockDataChallenge/IPTA_Challenge1_open/Challenge_Data/Dataset1/{psr_name}.tim'
psr = SingleBandPulsar.read_par_tim(par, tim, timing_package="pint", ephem="DE440", bipm_version="BIPM2019", clk="TT(BIPM2019)")

/fred/oz022/tkimpson/conda_envs/Argus3/lib/python3.12/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
2025-01-29 12:00:54.702 | WARNING  | pint.models.model_builder:__call__:229 - UNITS is not specified. Assuming TDB...
2025-01-29 12:00:54.769 | DEBUG    | pint.toa:__init__:1377 - No pulse number flags found in the TOAs
2025-01-29 12:00:54.770 | DEBUG    | pint.toa:apply_clock_corrections:2224 - Applying clock corrections (include_bipm = False)
2025-01-29 12:00:54.779 | INFO     | pint.observatory.topo_obs:clock_corrections:354 - Observatory axis requires no clock corrections.
2025-01-29 12:00:54.843 | DEBUG    | pint.toa:compute_TDBs:2270 - Computing TDB columns.
2025-01-29 12:00:54.843 | DEBUG    | pint.toa:compute_TDBs:2291 - Using EPHEM = DE440 for TDB calculation.
2025-01-29 12:00:54.863 | DEBUG    | pint.toa:get_TOAs:310 - Planet PosVels will be calculated.
2025

In [3]:
psr.radio_freqs

array([1440.14727355, 1440.14530109, 1440.13470477, 1440.11586383,
       1440.09035538, 1440.0595717 , 1440.02572611, 1439.99066746,
       1439.95663645, 1439.92537997, 1439.89868323, 1439.87782549,
       1439.86387679, 1439.8574873 , 1439.85892265, 1439.86816386,
       1439.88466503, 1439.9077524 , 1439.93610841, 1439.96847267,
       1440.00287701, 1440.03765874, 1440.07048111, 1440.09965343,
       1440.12295735, 1440.13922022, 1440.14693234, 1440.14596296,
       1440.13593551, 1440.11796914, 1440.09282138, 1440.06260608,
       1440.0287902 , 1439.99391408, 1439.95956353, 1439.92812656,
       1439.90082994, 1439.87952105, 1439.86480993, 1439.85782235,
       1439.85846688, 1439.86709317, 1439.88290985, 1439.90549331,
       1439.93339902, 1439.96548666, 1439.99978119, 1440.03459766,
       1440.06770642, 1440.09724265, 1440.12119794, 1440.1380746 ,
       1440.1466675 , 1440.14638829, 1440.13726117, 1440.11984761,
       1440.09540954, 1440.06543534, 1440.03200351, 1439.99698

The resulting `psr` object has some key instance attributes:

* `psr_residuals`. This is our data $\boldsymbol{Y}$
* `toa_diffs`. i.e. the $\Delta t$ value, which is required in the Kalman equations 
* `Mmat`. i.e. the design matrix corrections. See e.g. section 4.2 of  `explore_how_minnow_works.ipynb`

Note that, as expected, the length of `toa_diffs` is one less than the length of `residuals`

In [3]:
print(psr.residuals.shape)
print(psr.toa_diffs.shape)

(130,)
(129,)


Also note the shape of `Mmat`:

In [4]:
psr.Mmat.shape 

(130, 8)

**Question 2:** How to think about the elements of `MMat`? These are the timing model corrections. Should we think of these as free parameters to be inferred? Or are they parameters that we can take as known exactly a-priori? I feel like it should be the former, as "wiggles" in the GW that we are searching for could be caught in the "wiggles" of `MMat`. But then, do we really have 130 x 8 parameters per pulsar to search over?! In `explore_how_minnow_works.ipynb`, when we load the `.feather` files, the number of parameters is independent of the number of times, which agrees with my intution. I think I am missing an $\boldsymbol{\epsilon}$ vector somewhere, and `SingleBandPulsar` is returning $\boldsymbol{M} \boldsymbol{\epsilon}$


**Response to Question 2:** Ok here is what I think is going on. For a single pulsar, absent all other effects, the vector (length $N_{\rm obs}$) of timing residuals over the total observation period is given by $\boldsymbol{M} \boldsymbol{\delta \epsilon}$. Now $\boldsymbol{M}$ is the design matrix, with shape $N_{\rm obs}$ $\times$ $N_{\rm parameters}$. It is returned by the timing model fit and is just a big matrix of numbers. The vector $\boldsymbol{\delta \epsilon}$ has length $N_{\rm parameters}$. We want to infer these parameters. **More questions arise:**, physically, what are these parameters; what are the priors on these parameters, are there $N_{\rm parameters}$ timing parameters per pulsar (surely yes?)

**Addendum:** Some extra discoveries w.r.t Question 2. First the parameters that make up $\boldsymbol{\delta \epsilon}$ can be listed via `psr.fitpars`:

In [5]:
psr.fitpars

['Offset', 'PX', 'RAJ', 'DECJ', 'PMRA', 'PMDEC', 'F0', 'F1']

Second, the priors on these ephemeris parameters can just be some improper uniform prior, see end of section 7.1.1 of [Taylor 2021](https://arxiv.org/pdf/2105.13270). We may also ultimately be able to marginalise over these parameters by including them in the state...

### 2. Hidden states and transition matrix

Following `minnow`, the hidden state variables are 

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f} \right] $$

i.e. the **deviations from the spin-down parameters in the timing model fit**.

These variables evolve as,

$$ \frac{d \, \delta \phi}{dt} = \delta f$$

$$ \frac{d \, \delta f}{dt} = \delta \dot{f}$$

$$ \frac{d \, \delta \dot{f}}{dt} = 0 $$

(ignoring for now stochastic variations, i.e. Q-matrix corrections)

or, taking $\frac{d\boldsymbol{X}}{dt} = \boldsymbol{A} \boldsymbol{X}$, then 

$$\boldsymbol{A} = \begin{pmatrix}0 & 1 & 0 \\\ 0 & 0 &1 \\\ 0 & 0 & 0\end{pmatrix} $$

We can compute the transition matrix $\boldsymbol{F}_{\phi}$ by discretising in the usual way, i.e. $\boldsymbol{F}_{\phi} = \exp \left( \boldsymbol{A} \Delta t \right)$, where $\Delta t$ is the time difference between two consecutive TOAs, 

$$\boldsymbol{F}_{\phi} = \begin{pmatrix}1 & \Delta t & \Delta t^2/2  \\\ 0 & 1 &\Delta t \\\ 0 & 0 & 1\end{pmatrix} $$

This matrix is equivalent to that defined in https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L87 

### 3. Measurement matrix 

The measurement matrix $\boldsymbol{H}$ relates the states and the observations linearly, $\boldsymbol{Y} = \boldsymbol{H} \boldsymbol{X}$ (+ noise). The observations are just the timing residuals,

$$\boldsymbol{Y}(t_i) = \left [\delta t(t_i) \right ]$$

The timing residual is related to the state as 

$$ \delta t \left( t_i \right) = \frac{\delta \phi (t_i)}{f_0} + \boldsymbol{M} \boldsymbol{\epsilon}$$

where $f_0$ is the pulsar rotation frequency.

**Question 3: I guess we can just take $f_0$ as know exactly a-priori, or else just have it as a free parameter with a narrow prior**.

Assuming that my understanding at the end of Section 1 (Question 2) is correct, we can include the measurement effects as follows. First we augment the state-space

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f},\delta \epsilon_1,\dots,\delta \epsilon_p \right] $$

$$\boldsymbol{F} = \begin{pmatrix} \boldsymbol{F}_{\phi} & 0  \\\ 0 & \boldsymbol{I} \end{pmatrix} $$


where $\boldsymbol{F}_{\phi}$ is a block matrix, dimension 3x3, for $\delta \phi, \delta f, \delta \dot{f}$, as defined in Section 2, and $\boldsymbol{I}$ is an identity matrix, with dimension $p$, dimension of the design matrix. This matrix is equivalent to that defined in https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L7 (minus the DM corrections).


Second, we define the $\boldsymbol{H}$ matrix as


$$\boldsymbol{H}(t_i) = \begin{bmatrix} f_0^{-1}  & 0 & 0 & \boldsymbol{M}_1(t_i) & \dots & {M}_p(t_i) \end{bmatrix}$$

where ${M}_p(t_i)$ is given by the timing model fit. This is equivalent to https://github.com/meyers-academic/minnow/blob/main/src/minnow/signals.py#L53, minus the DM corrections.



### 4. Inclusion of stochasticity 


All of the above ignored the process noise and the measurement noise. Lets now bring these in. 


The dynamical state equations are 


$$ \frac{d \, \delta \phi}{dt} = \delta f + \xi(t, \sigma_{\phi})$$

$$ \frac{d \, \delta f}{dt} = \delta \dot{f}+ \xi(t, \sigma_{f})$$

$$ \frac{d \, \delta \dot{f}}{dt} = + \xi(t, \sigma_{\dot{f}}) $$


The transition and measurement matrices are unchanged, but we need to define a Q-matrix. For the above equations, I compute this to be:

$$Q = \begin{pmatrix}
\sigma_\phi^2\Delta t + \sigma_f^2\Delta t^3/3 + \sigma_{\dot{f}}^2\Delta t^5/20 & \sigma_f^2\Delta t^2/2 + \sigma_{\dot{f}}^2\Delta t^4/8 & \sigma_{\dot{f}}^2\Delta t^3/6 \\
\sigma_f^2\Delta t^2/2 + \sigma_{\dot{f}}^2\Delta t^4/8 & \sigma_f^2\Delta t + \sigma_{\dot{f}}^2\Delta t^3/3 & \sigma_{\dot{f}}^2\Delta t^2/2 \\
\sigma_{\dot{f}}^2\Delta t^3/6 & \sigma_{\dot{f}}^2\Delta t^2/2 & \sigma_{\dot{f}}^2\Delta t
\end{pmatrix}$$

which agrees (I think!) with [the Q-matrix defined in minnow](https://github.com/meyers-academic/minnow/blob/326dade647df22cadde807c1a5fbb48888c5dbe2/src/minnow/signals.py#L138) (ignoring the zero-entries corresponding to the $\delta \epsilon$ terms in the state).


The measurement matrix $\boldsymbol{R}$ is trivially just $\sigma_{\rm m}^2.$ We can either take this as known a-priori, or take it as a free parameter to be inferred. We may also consider EFAC, EQUAD, etc. here, see e.g. Equation 8 of https://arxiv.org/abs/1407.1838

### 5. Inclusion of GWs

The inclusion of the GW effects proceeds similarly to [PTA P3](https://arxiv.org/abs/2501.06990)

Specifically the $X$ state is augmented to include a factor $a(t)$

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f}, a, \delta \epsilon_1,\dots,\delta \epsilon_p \right] $$

The factor $a(t)$ evoles according to an Ornstein-Uhlenbeck process

$$ \frac{d \, a}{dt} = - \gamma_{\rm a} a + \xi(t, \sigma_{\rm a})$$

The covariance statistics of $\xi(t, \sigma_{\rm a})$ are givein in https://arxiv.org/abs/2501.06990

### 5. Summary



For the purposes of running state-space algorithms, we need to define the following:

* $\boldsymbol{X}$ : the state vector, dimension $n_X$
* $\boldsymbol{Y}$ : the observation vector, dimension $n_Y$
* $\boldsymbol{F}$ : the state transition matrix, dimension $n_X$ $\times$ $n_X$
* $\boldsymbol{H}$ : the measurement matrix, dimension $n_Y$ $\times$ $n_X$
* $\boldsymbol{Q}$ : the process noise covariance matrix, dimension $n_X$ $\times$ $n_X$
* $\boldsymbol{R}$ : the measurement noise covariance matrix, dimension $n_Y$ $\times$ $n_Y$



These components (for a single pulsar) are as follows:

$$\boldsymbol{X} = \left[\delta \phi, \delta f, \delta \dot{f},\delta \epsilon_1,\dots,\delta \epsilon_p \right] $$

$$\boldsymbol{Y}(t_i) = \left [\delta t(t_i) \right ]$$

$$\boldsymbol{F} = \begin{pmatrix} \boldsymbol{F}_{\phi} & 0  \\\ 0 & \boldsymbol{I} \end{pmatrix} $$

$$\boldsymbol{H}(t_i) = \begin{bmatrix} f_0^{-1}  & 0 & 0 & \boldsymbol{M}_1(t_i) & \dots & {M}_p(t_i) \end{bmatrix}$$

$$Q = \begin{pmatrix}
\sigma_\phi^2\Delta t + \sigma_f^2\Delta t^3/3 + \sigma_{\dot{f}}^2\Delta t^5/20 & \sigma_f^2\Delta t^2/2 + \sigma_{\dot{f}}^2\Delta t^4/8 & \sigma_{\dot{f}}^2\Delta t^3/6 \\
\sigma_f^2\Delta t^2/2 + \sigma_{\dot{f}}^2\Delta t^4/8 & \sigma_f^2\Delta t + \sigma_{\dot{f}}^2\Delta t^3/3 & \sigma_{\dot{f}}^2\Delta t^2/2 \\
\sigma_{\dot{f}}^2\Delta t^3/6 & \sigma_{\dot{f}}^2\Delta t^2/2 & \sigma_{\dot{f}}^2\Delta t
\end{pmatrix}$$


$$\boldsymbol{R} = \sigma_{\rm m}^2 $$



### Appendix

We now need to extend this to $N$ pulsars, sampled at different times, and also include the influecne of the GW via the $a(t)$ correction from paper 3. 


We are ultimately interested in the GW parameters. Can we marginalise over these parameters by adding them to the state evolution? See e.g. section 7.2.2 of [Taylor 2021](https://arxiv.org/pdf/2105.13270)

### A1. Summary of current thoughts on timing model parameters

Also shared the below with P.Meyers


Ok I think my head is a bit clearer on this now, having looked better at minnow, enterprise and PINT. Sharing a summary of some thoughts below.

1. Each pulsar has an M matrix with shape N_residuals x N_{ephemeris parameters} . This is provided by TEMPO/PINT 
2. Each pulsar has a vector \delta \epsilon , length N_{ephemeris parameters} . The product M M * \delta \epsilon gives the timing residuals (length N_residuals).
3. For a state-space representation, M "lives" in the measurement matrix H . That is, at the i-th timestep, we select the i-th row of M to construct H .
4. Similarly, \delta \epsilon "lives" in the state vector X . The deterministic evolution is zero (i.e. each element of \delta \epsilon is constant, with identity transition matrix). The variation in \delta \epsilon is captured by the Q-matrix. Importantly, this means that we do not need any priors on the components of \delta \epsilon 
5. In step 4, I think this is somewhat (or exactly?!) equivalent to marginalising over the parameters that we don't really care about. One could also imagine pulling  \delta \epsilon out of X and bringing it into the measurement part of the filter, and then using (e.g.) MCMC to estimate the value of \delta \epsilon .  But we don't really care about the values of \delta \epsilon , we want to claim evidence for a GW.
6. How we set the Q-matrix to allow for the variation in  \delta \epsilon will be important here. Maybe I have missed something, but I couldn't see this implemented in minnow yet? 